In [1]:
#Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import warnings
from sklearn import metrics
from sklearn.metrics import * 

plt.rc('font', size=12)
warnings.filterwarnings('ignore')

In [2]:
#Load data
#ESCAPE
escape = pd.read_csv("Preprocessed Data/EscapeAllData.csv", sep=",", index_col='DEIDNUM').sort_index() #all feature dataset
escapeSV = pd.read_csv("Preprocessed Data/EscapeAllDataSingleValue.csv", sep=",", index_col='DEIDNUM').sort_index() #all feature dataset
escapeLabels  = pd.read_csv("../Data/Original DataFrames/Labels.csv", sep=",", index_col='DEIDNUM').sort_index() #labels for prediction classes 

#BEST
#Only has discharge data + no hemo
best = pd.read_csv("Preprocessed Data/AllDataBEST.csv", sep=",", index_col='ID').sort_index() #all feature dataset
bestLabels  = pd.read_csv("Preprocessed Data/LabelsBEST.csv", sep=",", index_col='ID').sort_index() #labels for prediction classes 

#HF ACTION
#Only has discharge data + no hemo
hfaction = pd.read_csv("Preprocessed Data/AllDataHF-ACTION.csv", sep=",", index_col='ID').sort_index() #all feature dataset
hfactionLabels  = pd.read_csv("Preprocessed Data/LabelsHF-ACTION.csv", sep=",", index_col='ID').sort_index() #labels for prediction classes 

#Guide it
guide = pd.read_csv("Preprocessed Data/GUIDEAllData.csv", sep=",", index_col='ID').sort_index() #all feature dataset
guideSV = pd.read_csv("Preprocessed Data/GUIDEAllDataSingleValue.csv", sep=",", index_col='ID').sort_index() #all feature dataset
guideLabels  = pd.read_csv("Preprocessed Data/LabelsGUIDE-IT.csv", sep=",", index_col='ID').sort_index() #labels for prediction classes 

#Cardiogenic Shock
cardShock = pd.read_csv("../Data Validation/Cardiogenic Shock/Preprocessed Data/CardiogenicShock_AllData.csv", sep=",", index_col='ID').sort_index()
cardShockLabels = pd.read_csv("../Data Validation/Cardiogenic Shock/Original DataFrames/LabelsCardiogenicShock.csv", sep=",", index_col='ID').sort_index()

#Serial Cardiac
serial = pd.read_csv("../Data Validation/Serial Cardiac Caths/Preprocessed Data/SerialCardiac_AllData.csv", sep=",", index_col='ID').sort_index()
serialLabels = pd.read_csv("../Data Validation/Serial Cardiac Caths/Original DataFrames/LabelsSerialCardiac.csv", sep=",", index_col='ID').sort_index()


In [3]:
#Calculate ESCAPE Risk score
def calcESCAPEScore(data):
    lst = []
    origIdx = data.index
    idxName = data.index.name
    data = data.reset_index()
    data = data.drop(columns=[idxName])
    
    for idx in data.index:
        pt = data.loc[idx]
        scr = 0
        if pt.iloc[0] > 70: #Age
            scr += 1
        if pt.iloc[1] > 40: #BUN
            scr += 1
        if pt.iloc[1] > 90: #BUN
            scr += 1
        if pt.iloc[2] < 300: #Walk ft
            scr += 1
        if pt.iloc[3] < 130: #SOD
            scr += 1
        if pt.iloc[4] == 1.0 or pt.iloc[5] == 1.0: #CPR or MEVT
            scr += 2
        if pt.iloc[6] > 240: #DIUR Dose
            scr += 1
        if pt.iloc[7] == 0.0: #BETA blockers
            scr += 1
        if pt.iloc[8] > 500: #BNP
            scr += 1
        if pt.iloc[8] > 1300: #BNP
            scr += 3
            
        #associate score with prob of outcome
        prob = 0
        if scr == 0:
            prob = 0.033
        elif scr == 1:
            prob = 0.065
        elif scr == 2:
            prob = 0.123
        elif scr == 3:
            prob = 0.223
        elif scr == 4:
            prob = 0.368
        elif scr == 5:
            prob = 0.543
        elif scr == 6:
            prob = 0.708
        elif scr == 7:
            prob = 0.831
        else: #>= 8 
            prob = 0.909

        lst.append(prob)


    rskDF = pd.DataFrame(lst, columns=['ESCAPE'], index=origIdx)
    return rskDF

#OPTIMIZE-HF risk score calculation
def calcOPTIMIZEHF(data): 
    origIdx = data.index
    data = data.reset_index()
    ageDict = {20:0, 25:2, 30:3, 35:5, 40:6, 45:8, 50:9, 55:11, 60:13, 65:14, 70:16, 75:17, 80:19, 85:20, 90:22, 95:24}
    hrDict = {65:0, 70:1, 75:1, 80:2, 85:3, 90:4, 95:4, 100:5, 105:6, 110:6}
    sysBPDict = {50:22, 60:20, 70:18, 80:16, 90:14, 100:12, 110:10, 120:8, 130:6, 140:4, 150:2, 160:0}
    sodDict = {110:13, 115:11, 120:9, 125:7, 130:4, 135:2, 140:0, 145:2, 150:4, 155:6, 160:8, 165:10, 170:12}
    crtDict = {0.0:0, 0.5:2, 1.0:5, 1.5:7, 2.0:10, 2.5:12, 3.0:15, 3.5:17}
    #primary cause of admission is hf for all cohorts so score starts as 0
    
    lst = []
    for idx in data.index:
        pt = data.loc[idx]
        scr = 0

#         if pt.isnull().values.any():
#             prob = np.nan
            
#         else:
        age = pt["Age"]
        if age >= 95:
            scr += 24
        elif age <= 20:
            scr += 0
        elif np.isnan(age):
            pass
        else:
            ageNearest = 5 * round(age/5)
            scr += ageDict[ageNearest]

        hr = pt["HRTRT"]
        if hr >= 110:
            scr += 6
        elif hr <= 65:
            scr += 0
        elif np.isnan(hr):
            pass
        else:
            hrNearest = 5 * round(hr/5)
            scr += hrDict[hrNearest]

        bp = pt["BPSYS"]
        if bp >= 160:
            scr += 0
        elif bp <= 50:
            scr += 22
        elif np.isnan(bp):
            pass
        else:
            bpNearest = 10 * round(bp/10)
            scr += sysBPDict[bpNearest]

        sod = pt["SOD"]
        if np.isnan(sod):
            pass
        else:
            sodNearest = 5 * round(sod/5)
            scr += sodDict[sodNearest]

        crt = pt["CRT"]
        if crt >= 3.5:
            scr += 17
        elif np.isnan(crt):
            pass
        else:
            crtNearest = 0.5 * round(crt/0.5)
            scr += crtDict[crtNearest]

        if pt['EjF'] < 40.0:
            scr += 1

        scr = 5 * round(scr/5)

        #associate score with prob of outcome
        prob = 0
        if scr == 30:
            prob = 0.02
        elif scr == 35:
            prob = 0.03
        elif scr == 40:
            prob = 0.05
        elif scr == 45:
            prob = 0.06
        elif scr == 50:
            prob = 0.1
        elif scr == 55:
            prob = 0.18
        elif scr == 60:
            prob = 0.25
        elif scr == 65:
            prob = 0.4
        elif scr == 70:
            prob = 0.5
        elif scr == 75:
            prob = 0.7
        elif scr == 80:
            prob = 0.8
        elif scr == 85:
            prob = 0.85
        elif scr == 90:
            prob = 0.91
        elif scr == 95:
            prob = 0.96
        else: #== 100 
            prob = 0.97
    
        lst.append(prob)
        
    rskDF = pd.DataFrame(lst, columns=['OPTIMIZE-HF'], index=origIdx)
    return rskDF

#Calc effect risk score - 30 day mort
def calcEFFECT30Day(data):
    origIdx = data.index
    data = data.reset_index()
    
    lst = []
    for idx in data.index:
        pt = data.loc[idx]
        scr = 0

        scr += pt['Age']

        if pt['Resp'] <= 20:
            scr += 20
        elif pt['Resp'] >= 45:
            scr += 45
        elif np.isnan(pt['Resp']):
            pass
        else:
            scr += pt['Resp']

        bp = pt['BPSYS']
        if bp >= 180:
            scr -= 60
        elif bp >= 160 and bp <= 179:
            scr -= 55
        elif bp >= 140 and bp <= 159:
            scr -= 50
        elif bp >= 120 and bp <= 139:
            scr -= 45
        elif bp >= 100 and bp <= 119:
            scr -= 40
        elif bp >= 90 and bp <= 99:
            scr -= 35
        elif np.isnan(bp):
            pass
        else: #<90
            scr -= 30

        if pt['BUN'] >= 60:
            scr += 60
        elif np.isnan(pt['BUN']):
            pass
        else:
            scr += pt['BUN']

        if pt['SOD'] < 136:
            scr += 10

        if pt['StrokeTIA'] == 1.0:
            scr += 10

        if pt['COPD'] == 1.0:
            scr += 10

        if pt['HEPT'] == 1.0:
            scr += 25

        if pt['MALIG'] == 1.0:
            scr += 15

        if pt['HEM'] < 10:
            scr += 0
            
        #Calculate prob
        if scr <= 60:
            prob = 0.004
        elif scr >= 61 and scr <= 90:
            prob = 0.034
        elif scr >= 91 and scr <= 120:
            prob = 0.122
        elif scr >= 121 and scr <= 150:
            prob = 0.327
        else: #scr > 150:
            prob = 0.59
            
        lst.append(prob)
        
    rskDF = pd.DataFrame(lst, columns=['EFFECT 30 Day'], index=origIdx)
    return rskDF


#Calc effect risk score - 1 year mort
def calcEFFECT1Year(data):
    origIdx = data.index
    data = data.reset_index()
    
    lst = []
    for idx in data.index:
        pt = data.loc[idx]
        scr = 0

        scr += pt['Age']

        if pt['Resp'] <= 20:
            scr += 20
        elif pt['Resp'] >= 45:
            scr += 45
        elif np.isnan(pt['Resp']):
            pass
        else:
            scr += pt['Resp']

        bp = pt['BPSYS']
        if bp >= 180:
            scr -= 50
        elif bp >= 160 and bp <= 179:
            scr -= 45
        elif bp >= 140 and bp <= 159:
            scr -= 40
        elif bp >= 120 and bp <= 139:
            scr -= 35
        elif bp >= 100 and bp <= 119:
            scr -= 30
        elif bp >= 90 and bp <= 99:
            scr -= 25
        elif np.isnan(bp):
            pass
        else: #<90
            scr -= 20

        if pt['BUN'] >= 60:
            scr += 60
        elif np.isnan(pt['BUN']):
            pass
        else:
            scr += pt['BUN']

        if pt['SOD'] < 136:
            scr += 10

        if pt['StrokeTIA'] == 1.0:
            scr += 10

        if pt['COPD'] == 1.0:
            scr += 10

        if pt['HEPT'] == 1.0:
            scr += 35

        if pt['MALIG'] == 1.0:
            scr += 15

        if pt['HEM'] < 10:
            scr += 10
            
        #Calculate prob
        if scr <= 60:
            prob = 0.078
        elif scr >= 61 and scr <= 90:
            prob = 0.129
        elif scr >= 91 and scr <= 120:
            prob = 0.325
        elif scr >= 121 and scr <= 150:
            prob = 0.593
        else: #scr > 150:
            prob = 0.788
            
        lst.append(prob)
        
    rskDF = pd.DataFrame(lst, columns=['EFFECT 1 Year'], index=origIdx)
    return rskDF

def makeLabels(data, labels):
    lst = []
    idx = sorted(data.index)
    for i in idx:
        lab = labels.loc[i]
        lst.append(lab['Death'])
        
    return pd.DataFrame(lst, columns=['Death'],index=idx)

In [4]:
#Run all risk scores and save to CSV
#Dataset: ESCAPE
data = escape
dataSV = escapeSV #single value
lbls = escapeLabels
lblsSV = makeLabels(dataSV, lbls)
name = "ESCAPE"

#ESCAPE Risk Score
esRs = data.filter(['Age','BUN_D','SixFtWlk_D', 'SOD_D', 'CPR', 'MEVT', 'DIURDSE_D', 'BET_D', 'BNP_D'], axis=1)
res = calcESCAPEScore(esRs)
res['Real'] = lbls['Death']

#Save escape score
res.to_csv("Calculated Scores/" + name + "_escapeScore.csv")

#OPTIMIZE-HF
esRs = dataSV.filter(['Age','HRTRT','BPSYS', 'SOD', 'CRT', 'EjF'], axis=1)
res = calcOPTIMIZEHF(esRs)

#EFFECT 30 day
esRs = dataSV.filter(['Age','Resp','BPSYS', 'BUN', 'SOD', 'StrokeTIA', 'COPD', 'HEPT', 'MALIG', 'HEM'], axis=1)
res['EFFECT 30 Day'] = calcEFFECT30Day(esRs)

#EFFECT 1 year
esRs = dataSV.filter(['Age','Resp','BPSYS', 'BUN', 'SOD', 'StrokeTIA', 'COPD', 'HEPT', 'MALIG', 'HEM'], axis=1)
res['EFFECT 1 Year'] = calcEFFECT1Year(esRs)

res['Real'] = lblsSV['Death']
res.to_csv("Calculated Scores/" + name + "_optimizeEffectScore.csv")

res

,OPTIMIZE-HF,EFFECT 30 Day,EFFECT 1 Year,Real
DEIDNUM,,,,
72,0.06,0.122,0.593,1
72,0.10,0.327,0.593,1
81,0.06,0.122,0.593,0
81,0.06,0.122,0.325,0
86,0.97,0.122,0.325,0
...,...,...,...,...
99302,0.97,0.034,0.129,1
99912,0.97,0.034,0.129,0
99912,0.97,0.034,0.129,0


In [5]:
#Dataset: GUIDE
data = guide
dataSV = guideSV #single value
lbls = guideLabels
lblsSV = makeLabels(dataSV, lbls)
name = "GUIDE"

#ESCAPE Risk Score
data['CPR'] = np.nan
esRs = data.filter(['Age','BUN_D','SixFtWlk', 'SOD_D', 'CPR', 'MEVT', 'DIURDSE_B', 'BET_B', 'BNP_D'], axis=1)
res = calcESCAPEScore(esRs)
res['Real'] = lbls['Death']

#Save escape score
res.to_csv("Calculated Scores/" + name + "_escapeScore.csv")

#OPTIMIZE-HF
esRs = dataSV.filter(['Age','HRTRT','BPSYS', 'SOD', 'CRT', 'EjF'], axis=1)
res = calcOPTIMIZEHF(esRs)

#EFFECT 30 day
esRs = dataSV.filter(['Age','Resp','BPSYS', 'BUN', 'SOD', 'StrokeTIA', 'COPD', 'HEPT', 'MALIG', 'HEM'], axis=1)
res['EFFECT 30 Day'] = calcEFFECT30Day(esRs)

#EFFECT 1 year
esRs = dataSV.filter(['Age','Resp','BPSYS', 'BUN', 'SOD', 'StrokeTIA', 'COPD', 'HEPT', 'MALIG', 'HEM'], axis=1)
res['EFFECT 1 Year'] = calcEFFECT1Year(esRs)

res['Real'] = lblsSV['Death']
res.to_csv("Calculated Scores/" + name + "_optimizeEffectScore.csv")

res

,OPTIMIZE-HF,EFFECT 30 Day,EFFECT 1 Year,Real
ID,,,,
11,0.03,0.122,0.325,0
11,0.03,0.004,0.129,0
350,0.97,0.034,0.325,0
350,0.97,0.034,0.325,0
498,0.03,0.034,0.325,0
...,...,...,...,...
99676,0.97,0.034,0.129,0
99887,0.02,0.004,0.129,0
99887,0.97,0.004,0.078,0


In [6]:
#Dataset: BEST
data = best
dataSV = best #single value same
lbls = bestLabels
lblsSV = makeLabels(dataSV, lbls)
name = "BEST"

data['Resp'] = np.nan

#ESCAPE Risk Score
esRs = data.filter(['Age','BUN','SixFtWlk', 'SOD', 'CPR', 'MEVT', 'DIURDSE', 'BET', 'BNP'], axis=1)
res = calcESCAPEScore(esRs)
res['Real'] = lbls['Death']

#Save escape score
res.to_csv("Calculated Scores/" + name + "_escapeScore.csv")

#OPTIMIZE-HF
esRs = dataSV.filter(['Age','HRTRT','BPSYS', 'SOD', 'CRT', 'EjF'], axis=1)
res = calcOPTIMIZEHF(esRs)

#EFFECT 30 day
esRs = dataSV.filter(['Age','Resp','BPSYS', 'BUN', 'SOD', 'StrokeTIA', 'COPD', 'HEPT', 'MALIG', 'HEM'], axis=1)
res['EFFECT 30 Day'] = calcEFFECT30Day(esRs)

#EFFECT 1 year
esRs = dataSV.filter(['Age','Resp','BPSYS', 'BUN', 'SOD', 'StrokeTIA', 'COPD', 'HEPT', 'MALIG', 'HEM'], axis=1)
res['EFFECT 1 Year'] = calcEFFECT1Year(esRs)

res['Real'] = lblsSV['Death']
res.to_csv("Calculated Scores/" + name + "_optimizeEffectScore.csv")

res

,OPTIMIZE-HF,EFFECT 30 Day,EFFECT 1 Year,Real
ID,,,,
1.0,0.02,0.034,0.129,0
2.0,0.02,0.004,0.078,0
3.0,0.03,0.004,0.078,0
4.0,0.97,0.004,0.078,0
5.0,0.97,0.004,0.078,1
...,...,...,...,...
2704.0,0.03,0.004,0.078,0
2705.0,0.97,0.004,0.078,0
2706.0,0.03,0.004,0.129,1


In [7]:
#Dataset: HF ACTION
data = hfaction
dataSV = hfaction #single value
lbls = hfactionLabels
lblsSV = makeLabels(dataSV, lbls)
name = "HFACTION"

data['Resp'] = np.nan

#ESCAPE Risk Score
esRs = data.filter(['Age','BUN','SixFtWlk', 'SOD', 'CPR', 'MEVT', 'DIURDSE', 'BET', 'BNP'], axis=1)
res = calcESCAPEScore(esRs)
res['Real'] = lbls['Death']

#Save escape score
res.to_csv("Calculated Scores/" + name + "_escapeScore.csv")

#OPTIMIZE-HF
esRs = dataSV.filter(['Age','HRTRT','BPSYS', 'SOD', 'CRT', 'EjF'], axis=1)
res = calcOPTIMIZEHF(esRs)

#EFFECT 30 day
esRs = dataSV.filter(['Age','Resp','BPSYS', 'BUN', 'SOD', 'StrokeTIA', 'COPD', 'HEPT', 'MALIG', 'HEM'], axis=1)
res['EFFECT 30 Day'] = calcEFFECT30Day(esRs)

#EFFECT 1 year
esRs = dataSV.filter(['Age','Resp','BPSYS', 'BUN', 'SOD', 'StrokeTIA', 'COPD', 'HEPT', 'MALIG', 'HEM'], axis=1)
res['EFFECT 1 Year'] = calcEFFECT1Year(esRs)

res['Real'] = lblsSV['Death']
res.to_csv("Calculated Scores/" + name + "_optimizeEffectScore.csv")

res

,OPTIMIZE-HF,EFFECT 30 Day,EFFECT 1 Year,Real
ID,,,,
b'HFACT00001',0.03,0.004,0.788,0
b'HFACT00002',0.05,0.034,0.129,0
b'HFACT00003',0.02,0.004,0.078,0
b'HFACT00004',0.97,0.004,0.078,0
b'HFACT00005',0.03,0.004,0.129,0
...,...,...,...,...
b'HFACT02135',0.97,0.004,0.078,0
b'HFACT02136',0.03,0.004,0.129,1
b'HFACT02137',0.02,0.004,0.129,1


In [8]:
#Dataset: Cardiogenic Shock
data = cardShock
lbls = makeLabels(data, cardShockLabels)
name = "CardiogenicShock"

data = data.rename(columns={"HR": "HRTRT"})

data['CPR'] = np.nan
data['MEVT'] = np.nan
data['DIURDSE'] = np.nan
data['BNP'] = np.nan
data['Resp'] = np.nan

#ESCAPE Risk Score
esRs = data.filter(['Age','BUN','SixFtWlk', 'SOD', 'CPR', 'MEVT', 'DIURDSE', 'BET', 'BNP'], axis=1)
res = calcESCAPEScore(esRs)
res['Real'] = lbls['Death']

#Save escape score
res.to_csv("Calculated Scores/" + name + "_escapeScore.csv")

#OPTIMIZE-HF
esRs = data.filter(['Age','HRTRT','BPSYS', 'SOD', 'CRT', 'EjF'], axis=1)
res = calcOPTIMIZEHF(esRs)

#EFFECT 30 day
esRs = data.filter(['Age','Resp','BPSYS', 'BUN', 'SOD', 'StrokeTIA', 'COPD', 'HEPT', 'MALIG', 'HEM'], axis=1)
res['EFFECT 30 Day'] = calcEFFECT30Day(esRs)

#EFFECT 1 year
esRs = data.filter(['Age','Resp','BPSYS', 'BUN', 'SOD', 'StrokeTIA', 'COPD', 'HEPT', 'MALIG', 'HEM'], axis=1)
res['EFFECT 1 Year'] = calcEFFECT1Year(esRs)

res['Real'] = lbls['Death']
res.to_csv("Calculated Scores/" + name + "_optimizeEffectScore.csv")

res

,OPTIMIZE-HF,EFFECT 30 Day,EFFECT 1 Year,Real
ID,,,,
Z1002839,0.05,0.034,0.129,1
Z1002839,0.97,0.004,0.129,1
Z1002839,0.97,0.004,0.129,1
Z101592,0.05,0.122,0.325,1
Z101592,0.05,0.034,0.325,1
...,...,...,...,...
Z964794,0.03,0.034,0.129,1
Z964794,0.03,0.034,0.325,1
Z977261,0.97,0.004,0.078,1


In [9]:
#Dataset: Serial Cardiac
data = serial
lbls = makeLabels(data, serialLabels)
name = "SerialCardiac"

data = data.rename(columns={"HR": "HRTRT"})

data['CPR'] = np.nan
data['MEVT'] = np.nan
data['DIURDSE'] = np.nan
data['BNP'] = np.nan
data['Resp'] = np.nan

#ESCAPE Risk Score
esRs = data.filter(['Age','BUN','SixFtWlk', 'SOD', 'CPR', 'MEVT', 'DIURDSE', 'BET', 'BNP'], axis=1)
res = calcESCAPEScore(esRs)
res['Real'] = lbls['Death']

#Save escape score
res.to_csv("Calculated Scores/" + name + "_escapeScore.csv")

#OPTIMIZE-HF
esRs = data.filter(['Age','HRTRT','BPSYS', 'SOD', 'CRT', 'EjF'], axis=1)
res = calcOPTIMIZEHF(esRs)

#EFFECT 30 day
esRs = data.filter(['Age','Resp','BPSYS', 'BUN', 'SOD', 'StrokeTIA', 'COPD', 'HEPT', 'MALIG', 'HEM'], axis=1)
res['EFFECT 30 Day'] = calcEFFECT30Day(esRs)

#EFFECT 1 year
esRs = data.filter(['Age','Resp','BPSYS', 'BUN', 'SOD', 'StrokeTIA', 'COPD', 'HEPT', 'MALIG', 'HEM'], axis=1)
res['EFFECT 1 Year'] = calcEFFECT1Year(esRs)

res['Real'] = lbls['Death']
res.to_csv("Calculated Scores/" + name + "_optimizeEffectScore.csv")

res

,OPTIMIZE-HF,EFFECT 30 Day,EFFECT 1 Year,Real
ID,,,,
Z103426,0.97,0.004,0.078,0
Z103426,0.97,0.004,0.078,0
Z103426,0.97,0.004,0.078,0
Z103697,0.05,0.034,0.325,0
Z103697,0.02,0.004,0.129,0
...,...,...,...,...
Z958018,0.02,0.034,0.129,0
Z958018,0.03,0.004,0.078,0
Z98220,0.02,0.004,0.078,0
